Todo:
- Add all prediction apps, update URLs below
- Ensure prediction apps have clear visualizations
- Double check input/output files and example output format are correct!
- AntibodyProfiler: Accept HLT format PDB files, antigen chain
- Stress-test servers, spin up capacity before workshop

Nice to have:
- Add previously computed examples to visualize on BioLib.com
- New BioLib app: chotia_to_hlt for choosing own target and antibody framework
- Antibody liability app
- Leaderboard
- "Master" BioLib app implementing all functionality in one app (easier to parallelize)

## De novo generation of antibody binders with RFantibody
[![colab.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/mhoie/workshop/blob/main/workshop.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mhoie/workshop/blob/main/workshop.ipynb)

In this notebook, you may choose your own antibody framework and target protein structure, and design novel antibody binders. This workflow has been shown to generate weak antibody binders in the μM to nM range, with up to ~5-10% experimental success rates for some degree of binding.

---

Antibody therapeutics represent a substantial market (approximately $550M USD) with tremendous potential for treating various diseases. Traditional approaches to antibody discovery are slow and laborious, typically involving immunizing mice or screening random libraries. 

This notebook implements the RFantibody pipeline for structure-based design of de novo antibodies against a chosen target.

It takes two inputs:
- i) An input antibody framework (e.g. hu-4D5-8_Fv.pdb - a humanized single-domain antibody already approved in two FDA therapies)
- ii) A target protein of interest (e.g. respiratory syncytial virus (RSV) protein) with binding site (epitope residues)

And runs the following three methods:
1. **De novo design of an antibody backbone targeting a protein of interest** - using an antibody-finetuned version of RFdiffusion ([Nature paper](https://www.nature.com/articles/s41586-023-06415-8))
2. **Design of the CDR loop residues** - using ProteinMPNN ([Science paper](https://www.science.org/doi/10.1126/science.add2187))
3. **Filtering designs on predicted structure 'self-consistency'** - using an antibody-finetuned version of RoseTTAFold2 ([Preprint](https://www.biorxiv.org/content/10.1101/2023.05.24.542179v1)), shown to correlate with significantly improved experimental success rates.

The RFantibody pipeline itself is described in detail in [this preprint](https://www.biorxiv.org/content/10.1101/2024.03.14.585103v2).

Advantages:
- Designs novel antibodies binding a target protein
- Can target most epitope binding region of interest (preferring structured regions)
- Focuses on designing antibody CDR loops (main residues determining binding)
- Designs may be filtered by "self-cosistency" of predicted structures, correlating with experimental success rates

Current limitations:
- Generated antibodies at best tend to be weak binders (low binding affinities in μM to nM range)
- Often low experimental success rates (~5-10% for some degree of binding) - heavily dependent on filtering
- No screening for e.g. human immunogenicity

## Pre-requisites for this workshop

The only pre-requisites for this workshop are the following:
- i) Register a BioLib account on [https://biolib.com/sign-up](https://biolib.com/sign-up), for running RFantibody jobs (requires a GPU) in the cloud.
- ii) An antibody PDB and target protein structure PDB. Examples are provided below, but you may also provide your own following the instructions below.

#### i) Install pybiolib and login to biolib.com

In [1]:
# Download BioLib
!pip install --quiet --upgrade pybiolib

In [2]:
# Login with BioLib
import biolib
biolib.login()

2025-03-25 17:16:49,864 | INFO : Already signed in


#### ii) Set input antibody and target PDB
We're working with two primary input files for RFAntibody:

1. **framework_pdb**: `hu-4D5-8_Fv.pdb` - A humanized single domain antibody (VHH) framework
2. **target_pdb**: `rsv_site3.pdb` - The target protein (RSV site 3) of interest
3. List of hotspot residues defining our epitope (target binding site)

You may also choose your own antibody framework and target PDB, prepared with this script:
https://github.com/RosettaCommons/RFantibody?tab=readme-ov-file#input-preparation

Let's verify that these files exists.

In [3]:
# Download input files if not already present
!wget --no-verbose -nc "https://raw.githubusercontent.com/mhoie/workshop/refs/heads/main/example_input/hu-4D5-8_Fv.pdb"
!wget --no-verbose -nc "https://raw.githubusercontent.com/mhoie/workshop/refs/heads/main/example_input/rsv_site3.pdb"

In [4]:
import os
framework_pdb = "hu-4D5-8_Fv.pdb"
target_pdb = "rsv_site3.pdb"
hotspot_res = "[T305,T456]"
print(f"Framework PDB exists: {os.path.exists(framework_pdb)}")
print(f"Target PDB exists: {os.path.exists(target_pdb)}")
print(f"(If these are missing, please download from https://github.com/mhoie/bioit-rfantibody before proceeding)")

Framework PDB exists: True
Target PDB exists: True
(If these are missing, please download from https://github.com/mhoie/bioit-rfantibody before proceeding)


## Step 1 of 4: Generate antibody-antigen docking pose, with [RFDiffusion (antibody-finetuned)](https://biolib.com/BioLibDevelopment/prediction-app/)
*Estimated time: ~2-3 minutes*

This step takes the input antibody framework and target protein, and designs the 3D structure of new CDR loops in interaction with the target protein (antibody-antigen docking pose). The CDR loops will be generated as backbones only (no residues), with the actual residues to be determined in the next step.

#### [Input parameters](https://biolib.com/BioLibDevelopment/prediction-app/)

Already set above:
- **framework_pdb**: Path to the antibody framework we're using (e.g. hu-4D5-8_Fv.pdb)
- **target_pdb**: Path to the target protein structure (e.g. rsv_site3.pdb)
- **hotspot_res**: List of hotspot residues defining our epitope (target binding site)

New parameters:
- **design_loops**: Dictionary mapping each CDR loop to a range of allowed loop lengths
  - L1, L2, L3: Light chain CDR loops
  - H1, H2, H3: Heavy chain CDR loops
  - Numbers specify length ranges (e.g., L1:8-13 means loop L1 can be 8-13 residues long)
  - Example: `[L1:8-13,L2:7,L3:9-11,H1:7,H2:6,H3:5-13]`
- **num_designs**: Number of designs to generate (20)


In [31]:
# Input parameters (antibody framework and target PDBs are set above)
design_loops = "[L1:8-13,L2:7,L3:9-11,H1:7,H2:6,H3:5-13]"
num_designs = 1

# Run RFdiffusion through BioLib
app_rfdiff = biolib.load('BioITWorkshop/RFDiffusionAntibody')  # Replace with actual RFantibody app ID
job_rfdiff = app_rfdiff.start(
    target_pdb=target_pdb,
    framework_pdb=framework_pdb,
    hotspot_res=hotspot_res,
    design_loops=design_loops,
    num_designs=num_designs,
)

2025-03-25 16:15:56,983 | INFO : Loaded project BioITWorkshop/RFDiffusionAntibody:0.0.2
2025-03-25 16:16:01,125 | INFO : View the result in your browser at: https://biolib.com/results/50b629d4-2b62-49ce-b8d1-733fb4038bb7/


#### Wait for RFdiffusion output files (~2-3 minutes)
The RFdiffusion step generates PDB files containing the antibody framework with designed CDR loops docked to the target protein. At this stage, the CDR loops have backbone structures but no amino acid sequences yet!

Main output file:
- output/rfdiffusion/_0.pdb - Antibody PDB backbone (N, Ca, C, O atoms only), lacking the CDR loop residues (which will be predicted in the next step)

Example output format:
```pdb
ATOM      1  N   GLU H   1      23.793  -8.718 -21.757  1.00  1.00

ATOM      2  CA  GLU H   1      23.755  -8.421 -20.330  1.00  1.00

ATOM      3  C   GLU H   1      23.563  -6.931 -20.082  1.00  1.00

ATOM      4  O   GLU H   1      23.856  -6.105 -20.947  1.00  1.00

ATOM      5  N   VAL H   2      22.855  -6.630 -18.891  1.00  1.00
```

In [32]:
# Try to save job output files
#job_rfdiff = biolib.get_job("ff4b0442-acb9-4c44-95fe-28e87b692e7b") # DEBUG
assert job_rfdiff.get_status() == "completed", "Job still in progress..."
job_rfdiff.save_files("output/rfdiffusion", skip_file_if_exists=True)
!tree output/rfdiffusion

2025-03-25 16:29:28,326 | INFO : Saving 4 files to output/rfdiffusion...
output/rfdiffusion
├── _0.pdb
├── _0.trb
└── traj
    ├── _0_Xt-1_traj.pdb
    └── _0_pX0_traj.pdb

2 directories, 4 files


## Step 2 of 4: Design binding CDR loop residues with [ProteinMPNN](https://biolib.com/BioLibDevelopment/prediction-app/)
*Estimated time: <1 minute*

The second step takes the docks generated by RFdiffusion and assigns amino acid sequences to the CDR loops using ProteinMPNN. We use the base version of ProteinMPNN (not an antibody-finetuned model) with a wrapper script that focuses on designing just the CDR loops.

#### [Input parameters](https://biolib.com/BioLibDevelopment/prediction-app/)
- **pdb**: Directory containing the previous RFdiffusion output PDB files, or a single PDB file

In [6]:
# Input directory
input_dir = "output/rfdiffusion"  # Using the output from RFdiffusion

# Run ProteinMPNN
app_mpnn = biolib.load('BioITWorkshop/ProteinMPNNAb')
job_mpnn = app_mpnn.start(
    pdb=input_dir,
    num_seqs_per_struct=3
)

2025-03-25 17:17:16,917 | INFO : Loaded project BioITWorkshop/ProteinMPNNAb:0.0.3
2025-03-25 17:17:20,157 | INFO : View the result in your browser at: https://biolib.com/results/f1cac0f6-1bc5-4018-a18d-08639403f8a8/


#### Wait for ProteinMPNN output files (<1 minute)
ProteinMPNN outputs PDB files with the same structures as the input but with amino acid sequences designed for the CDR loops. By default, it provides one sequence per input structure.

Output files:
- _0_dldesign_0.pdb (antibody structure with predicted CDR residues)
- _0_dldesign_1.pdb (antibody structure with predicted CDR residues)
- ... etc


Example output:
```pdb
ATOM      1  N   GLU H   1      23.793  -8.718 -21.757  1.00  0.00
ATOM      2  CA  GLU H   1      23.755  -8.421 -20.330  1.00  0.00
ATOM      3  C   GLU H   1      23.563  -6.931 -20.082  1.00  0.00
ATOM      4  O   GLU H   1      23.856  -6.105 -20.947  1.00  0.00
ATOM      5  N   VAL H   2      22.855  -6.630 -18.891  1.00  0.00
ATOM      6  CA  VAL H   2      22.864  -5.216 -18.533  1.00  0.00
```

In [10]:
# Try to save job output files
if job_mpnn.is_pending():
    print("Job is still running...")
else:
    job_mpnn.save_files("output/proteinmpnn", skip_file_if_exists=True)
    !tree output/proteinmpnn

2025-03-25 17:19:39,104 | INFO : Saving 3 files to output/proteinmpnn...
Skipping output/proteinmpnn/_0_dldesign_0.pdb as a file with that name already exists locally.
Skipping output/proteinmpnn/_0_dldesign_1.pdb as a file with that name already exists locally.
Skipping output/proteinmpnn/_0_dldesign_2.pdb as a file with that name already exists locally.
output/proteinmpnn
├── _0_dldesign_0.pdb
├── _0_dldesign_1.pdb
└── _0_dldesign_2.pdb

1 directory, 3 files


## Step 3 / 4: Filter designs for predicted structure self-consistency, with [RosettaFold2 antibody fine-tuned](https://biolib.com/BioLibDevelopment/prediction-app/)
*Estimated time: ~1-2 minutes*

The final step uses an antibody-finetuned version of RoseTTAFold2 (RF2) to predict the structure of the designed sequences and assess whether RF2 is confident that the sequence will bind as designed.

The RFantibody protocol recommends filtering on the following metrics, shown to lead to an up to 10X improvement in experimental success rates.
- RF2 predicted alignment error (pAE) < 10
- RMSD between design and RF2 prediction < 2Å for the CDRs

### [Input parameters](https://biolib.com/BioLibDevelopment/prediction-app/)

- **input.pdb_dir**: Directory containing the PDB files from ProteinMPNN
- **num_recycles**: Number of recycling iterations in the model (default: 10)
- **hotspot**: Percentage of hotspots provided to the model (default: 10%)

In [12]:
# Input directory
input_dir = "output/proteinmpnn"  # Using the output from ProteinMPNN

# Run RosettaFold2
app_rf2 = biolib.load('BioITWorkshop/RF2Antibody')  # Replace with actual app ID
job_rf2 = app_rf2.start(
    pdb=input_dir,
    num_recycles=3,
)

2025-03-25 17:23:38,881 | INFO : Loaded project BioITWorkshop/RF2Antibody:0.0.3
2025-03-25 17:23:41,251 | INFO : View the result in your browser at: https://biolib.com/results/8b350a73-fba9-4635-a195-4d788340ca33/


#### Wait for RosettaFold2 output files (1-2 minutes)
RosettaFold2 predicts the structure of the designed antibodies and provides confidence metrics. We can use these to filter for promising designs.

Output files:
- scores.csv - Predicted structural quality scores for filtering of designs
- _0_dldesign_0.pdb - Predicted structure of design 0
- _0_dldesign_1.pdb - Predicted structure of design 1
- ... etc

Example output scores.tsv:
```csv
interaction_pae,pae,    pred_lddt,  target_aligned_antibody_rmsd, ..., framework_aligned_cdr_rmsd, ...
8.07,           8.77,   0.9,        11.53,                        ..., 2.18,                       ...
7.52,           8.19,   0.89,       18.97,                        ..., 2.4,                        ...
8.47,           9.15,   0.9,        10.8,                        ...,  2.35,                       ...

```

Of these, our target values are:
- Predicted alignment error (pae) below 10
- Framework aligned CDR rmsd (framework_aligned_cdr_rmsd) below 2.00

In [13]:
# Try to save job output files
if job_rf2.is_pending():
    print("Job is still running...")
else:
    job_rf2.save_files("output/rosettafold2", skip_file_if_exists=True)
    !tree "output/rosettafold2"

2025-03-25 17:30:34,829 | INFO : Saving 4 files to output/rosettafold2...
Skipping output/rosettafold2/_0_dldesign_2.pdb as a file with that name already exists locally.
Skipping output/rosettafold2/_0_dldesign_1.pdb as a file with that name already exists locally.
Skipping output/rosettafold2/_0_dldesign_0.pdb as a file with that name already exists locally.
Skipping output/rosettafold2/scores.csv as a file with that name already exists locally.
output/rosettafold2
├── _0_dldesign_0.pdb
├── _0_dldesign_1.pdb
├── _0_dldesign_2.pdb
└── scores.csv

1 directory, 4 files


## Step 4 of 4: Assess designs for pharmaceutical liabilities (AntibodyProfiler)

*Estimated time: <1 minute*

Here, we will use our in-house AntibodyProfiler tool to assess our antibody PDBs for how closely they relate to molecular properties of already approved therapeutic antibodies. Antibody designs outside this range may indicate higher risk of pharmaceutical liabilities, providing another early filtering step before designs are sent to the lab for experimental validation.

We recommend selecting designs within 2-sigma deviation of approved antibodies for the following metrics:
- Total CDR length between X and Y 
- CDR3 heavy chain length between X and Y 
- Patches of surface hydrophobicity in CDR vicinity (PSH) between X and Y 
- Patches of positive charge in CDR vicinity (PPC) between X and Y
- Patches of negative charge in CDR vicinity (PNC) between X and Y 
- Structural fragment variable charge symmetry parameter (SFvCSP) between X and Y

### [Input parameters](https://biolib.com/BioLibDevelopment/prediction-app/)

- **input.pdb_dir**: Directory containing the PDB files from ProteinMPNN
- **num_recycles**: Number of recycling iterations in the model (default: 10)
- **hotspot**: Percentage of hotspots provided to the model (default: 10%)

In [14]:
# Input directory
input_dir = "output/rosettafold2"  # Using the output from RosettaFold2

# Run RosettaFold2
app_abprofiler = biolib.load('BioLibDevelopment/AntibodyProfiler') # TODO - push to main branch so the last version is used
app_abprofiler = app_abprofiler.start(
    pdb=input_dir, # DEBUG - need to add automatic csv file generation for dir input, accepting hlt format
)

2025-03-25 17:41:21,718 | INFO : Loaded project BioLibDevelopment/AntibodyProfiler:0.0.32
2025-03-25 17:41:26,327 | INFO : View the result in your browser at: https://biolib.com/results/60efe401-0ca8-4fc3-8237-e8089eab16b7/


#### Wait for AntibodyProfiler output files (<1 minute)
AntibodyProfiler calculates developability metrics compared to approved therapeutic antibodies already on the market, giving warning flags for input PDBs outside this range.

Main output file:
- results.csv (Summary metrics compared to therapeutically approved antibodies for each PDB)

```csv
pdb_name,       cdrh3_length,total_cdr_length,flag_cdrh3,flag_total_cdr,psh_value,flag_psh
_0_dldesign_2,  13,          35,              none,       yellow,       116.90, none
_0_dldesign_1,  13,          35,              none,       yellow,       90.53,  none
_0_dldesign_0,  13,          35,              none,       yellow,       149.84, none
```

Of these, our target values are:
- CDR3 heavy chain length (cdrh3_length): 5-23
- CDR length (total_cdr_length): 39-60
- Patches of surface hydrophobicity in CDR vicinity (psh_value): 30-150

Values outside this range will be flagged with a color, depending on where they fall in the distribution of human therapeutic antibodies.

In [48]:
# Try to save job output files
if app_abprofiler.is_pending():
    print("Job is still running...")
else:
    job_abprofiler.save_files("output/abprofiler", skip_file_if_exists=True)
    !tree output/abprofiler

2025-03-25 16:57:44,984 | INFO : Saving 24 files to output/abprofiler...
output/abprofiler
├── _0_dldesign_0
│   ├── _0_dldesign_0.pdb
│   ├── _0_dldesign_0_CDRH3_length.png
│   ├── _0_dldesign_0_Human_CDRH3_length.png
│   ├── _0_dldesign_0_Human_CDR_length.png
│   ├── _0_dldesign_0_Human_PSH_distribution.png
│   ├── _0_dldesign_0_PSH_distribution.png
│   └── _0_dldesign_0_Total_CDR_length.png
├── _0_dldesign_1
│   ├── _0_dldesign_1.pdb
│   ├── _0_dldesign_1_CDRH3_length.png
│   ├── _0_dldesign_1_Human_CDRH3_length.png
│   ├── _0_dldesign_1_Human_CDR_length.png
│   ├── _0_dldesign_1_Human_PSH_distribution.png
│   ├── _0_dldesign_1_PSH_distribution.png
│   └── _0_dldesign_1_Total_CDR_length.png
├── _0_dldesign_2
│   ├── _0_dldesign_2.pdb
│   ├── _0_dldesign_2_CDRH3_length.png
│   ├── _0_dldesign_2_Human_CDRH3_length.png
│   ├── _0_dldesign_2_Human_CDR_length.png
│   ├── _0_dldesign_2_Human_PSH_distribution.png
│   ├── _0_dldesign_2_PSH_distribution.png
│   └── _0_dldesign_2_Total_CDR_le

## Conclusion

This notebook has demonstrated the complete RFantibody pipeline for structure-based design of de novo antibodies. The workflow consists of three main steps:

1. **RFdiffusion (antibody fine-tuned)**: Generating antibody-target docking poses with designed CDR loop structures
2. **ProteinMPNN (antibody fine-tuned)**: Designing amino acid sequences for the CDR loops
3. **RosettaFold2 (antibody fine-tuned)**: Filtering designs based on predicted structure quality
4. **AntibodyProfiler**: Further selection of designs based on similarity to therapeutically approved antibodies

This computational pipeline can generate designs with a success rate of approximately 5-10% for some degree of binding to the target. Further experimental validation and optimization is likely to be required to improve binding affinity and other pharmaceutical properties.

For larger scale antibody design campaigns, we recommended to generating thousands of designs to increase the chances of finding high-quality binders, as the current filtering metrics are still highly limited.